In [58]:
import speechbrain as sb
import sqltables
from pathlib import Path
import json
from tqdm.notebook import tqdm
import sqltables

db = sqltables.Database()

In [2]:
data_dir = Path("/data/nst")

In [3]:
audio_files = list(data_dir.rglob("*.wav"))

In [107]:
def extract_audio_path_components(path, data_dir):
    prefix = "/".join(path.relative_to(data_dir).parts[:3])
    recording = path.parts[-2].lower()
    audio_file = path.name
    return prefix, recording, audio_file, str(path)

audio_tab = db.create_table(rows=[extract_audio_path_components(x, data_dir) for x in audio_files], 
                            column_names=["prefix", "recording", "audio_file", "path"])
audio_tab

|prefix|recording|audio\_file|path|
|-|-|-|-|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512142\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512142\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512261\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512261\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512205\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512205\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512244\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512244\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512126\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512126\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512296\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512296\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512301\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512301\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512304\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512304\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512163\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512163\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512271\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512271\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512149\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512149\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512084\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512084\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512176\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512176\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512291\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512291\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512186\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512186\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512065\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512065\.wav\'|
|...|...|...|...|


In [6]:
transcript_files = list(data_dir.rglob("data/*/*/*/*.spl"))
len(transcript_files)

13026

In [78]:
def read_transcript(transcript_path, data_dir):
    prefix = "/".join(transcript_path.relative_to(data_dir).parts[:3])
    recording = transcript_path.stem
    with open(transcript_path) as f:
        while True:
            line = f.readline()
            if line == "[Info states]\n":
                break
            if line == "":
                return []
        speaker_info = {}
        while True:
            line = f.readline()
            if line == "\n" or line == "":
                break
            _, data = line.split("=", 2)
            cols = data.split(">-<")
            speaker_info[cols[0]] = cols[1]
        while True:
            line = f.readline()
            if line == "[Record states]\n":
                break
            if line == "":
                return []
        lines = []
        while True:
            line = f.readline()
            if line == "\n" or line == "":
                break
            lines.append(line)
    cols = [x.split(">-<") for x in lines]
    data = [(prefix, recording, speaker_info, x[5].lower(), x[2]) for x in cols if len(x) > 5]
    return data

In [79]:
read_transcript(transcript_files[0], data_dir)

[('train/Stasjon6/060700',
  'r5650512',
  {'Speaker ID': '509',
   'Name': '*******',
   'Age': '27',
   'Sex': 'Male',
   'Region of Youth': 'Vestjylland',
   'Region of Dialect': 'Vestjylland',
   'Sheet_ID': '509'},
  'u0512001.wav',
  '( ... tavshed under denne indspilning ...)'),
 ('train/Stasjon6/060700',
  'r5650512',
  {'Speaker ID': '509',
   'Name': '*******',
   'Age': '27',
   'Sex': 'Male',
   'Region of Youth': 'Vestjylland',
   'Region of Dialect': 'Vestjylland',
   'Sheet_ID': '509'},
  'u0512002.wav',
  'Tester en to tre fire fem seks syv otte.'),
 ('train/Stasjon6/060700',
  'r5650512',
  {'Speaker ID': '509',
   'Name': '*******',
   'Age': '27',
   'Sex': 'Male',
   'Region of Youth': 'Vestjylland',
   'Region of Dialect': 'Vestjylland',
   'Sheet_ID': '509'},
  'u0512003.wav',
  'Adskillige af de huse, som kommunen byggede sidste år, er blevet beskadiget under den sidste tids stormvejr.'),
 ('train/Stasjon6/060700',
  'r5650512',
  {'Speaker ID': '509',
   'Name':

In [80]:
transcripts = [row for fname in tqdm(transcript_files) for row in read_transcript(fname, data_dir)]

  0%|          | 0/13026 [00:00<?, ?it/s]

In [81]:
transcripts[2]

('train/Stasjon6/060700',
 'r5650512',
 {'Speaker ID': '509',
  'Name': '*******',
  'Age': '27',
  'Sex': 'Male',
  'Region of Youth': 'Vestjylland',
  'Region of Dialect': 'Vestjylland',
  'Sheet_ID': '509'},
 'u0512003.wav',
 'Adskillige af de huse, som kommunen byggede sidste år, er blevet beskadiget under den sidste tids stormvejr.')

In [82]:
transcript_rows = [(prefix, recording, audio_file, text) 
                   for (prefix, recording, _,audio_file, text) in transcripts]
transcript_tab = db.create_table(rows=set(transcript_rows), column_names=["prefix", "recording", "audio_file", "text"])
transcript_tab

|prefix|recording|audio\_file|text|
|-|-|-|-|
|\'train\/Stasjon03\/180700\'|\'r5650013\'|\'u0013194\.wav\'|\'Kammeradvokaten slog i går fast\, at alle regler i den forudgående arkitektkonkurrence er fulgt korrekt\.\'|
|\'train\/Stasjon02\/100800\'|\'r5650032\'|\'u0032013\.wav\'|\'otteogtres tusind fire hundrede og fireogfirs\'|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110049\'|\'u0049713\.wav\'|\'Tværtimod skal vi prøve at samarbejde og finde ud af\, hvilke lande der skal lave hvad i de militære operationer\.\'|
|\'train\/Stasjon02\/020800\'|\'r5650063\'|\'u0063295\.wav\'|\'makro\'|
|\'train\/Stasjon6\/020800\'|\'r5650554\'|\'u0554016\.wav\'|\'seksoghalvfjerds tusind et hundrede og fireoghalvfjerds\'|
|\'da\_0611\_test\/Stasjon03\/051000\'|\'r6110010\'|\'u0010474\.wav\'|\'fodboldturnering fodboldturnering fodboldturnering\'|
|\'da\_0611\_test\/Stasjon05\/270900\_b\'|\'r6110006\'|\'u0006482\.wav\'|\'F O R L O V E D E \'|
|\'train\/Stasjon03\/080800\'|\'r5650281\'|\'u0281035\.wav\'|\'Nogle af de sjoveste og mest underholdende filmfejl opstår på grund af manglende konsekvens og dårlig research\.\'|
|\'train\/Stasjon01\/280700\'|\'r5650127\'|\'u0127078\.wav\'|\'fem         fire         to         fire         otteogtres         seks hundrede og nioghalvfems\'|
|\'train\/Stasjon02\/040800\'|\'r5650173\'|\'u0173196\.wav\'|\'Han træner ikke længere så meget\, men derimod hårdere og i kortere intervaller\.\'|
|\'train\/Stasjon03\/260900\'|\'r5650036\'|\'u0036201\.wav\'|\'Selv om han først satte fødder på det store kontinent i en alder af seksten år\,\\\\Komma er det der\,\\\\Komma han føler sig hjemme\.\\\\Punktum\'|
|\'train\/Stasjon03\/130700\'|\'r5650044\'|\'u0044160\.wav\'|\'Dernæst gælder det de eksklusivaftaler\, som visse leverandører har måtte acceptere at skrive under på\.\'|
|\'da\_0611\_test\/Stasjon05\/081000\_a\'|\'r6110038\'|\'u0038736\.wav\'|\'Vi glemmer det aldrig nogensinde\.\'|
|\'train\/Stasjon4\/020800\'|\'r5650474\'|\'u0474113\.wav\'|\'Kommunen modtog en såkaldt erkenderdom ved laveste instans uden retslig prøvelse\, og jeg fik mine penge tilbage\.\'|
|\'da\_0611\_test\/Stasjon05\/260900\_c\'|\'r6110010\'|\'u0010134\.wav\'|\'Så tager han sig vældigt sammen\.\'|
|\'train\/Stasjon02\/100800\'|\'r5650038\'|\'u0038084\.wav\'|\'Studiet er som nævnt at skaffe menneskeligt materiale til senere fuldtidsansættelse\, og her er undervisning et lige så vægtigt element som forskning\.\'|
|...|...|...|...|


In [83]:
speaker_info_rows = set((prefix, recording, speaker_info["Speaker ID"], speaker_info["Age"], speaker_info["Sex"])
                     for (prefix, recording, speaker_info, audio_file, text) in transcripts)

In [84]:
speaker_info_tab = db.create_table(rows=speaker_info_rows, column_names=["prefix", "recording", "speaker_id", "age", "sex"])
speaker_info_tab

|prefix|recording|speaker\_id|age|sex|
|-|-|-|-|-|
|\'da\_0611\_test\/Stasjon05\/240900\'|\'r6110012\'|\'3\'|\'25\'|\'Male\'|
|\'train\/Stasjon02\/020800\'|\'r5650189\'|\'182\'|\'67\'|\'Male\'|
|\'train\/Stasjon02\/200700\'|\'r5650057\'|\'152\'|\'65\'|\'Male\'|
|\'train\/Stasjon4\/240700\'|\'r5650420\'|\'314\'|\'36\'|\'Female\'|
|\'train\/Stasjon01\/240700\'|\'r5650046\'|\'41\'|\'56\'|\'Female\'|
|\'train\/Stasjon01\/030800\'|\'r5650020\'|\'16\'|\'26\'|\'Female\'|
|\'train\/Stasjon02\/030800\'|\'r5650182\'|\'175\'|\'58\'|\'Female\'|
|\'da\_0611\_test\/Stasjon03\/051000\'|\'r6110046\'|\'44\'|\'21\'|\'Male\'|
|\'train\/Stasjon4\/120700\'|\'r5650410\'|\'305\'|\'46\'|\'Male\'|
|\'train\/Stasjon4\/140800\'|\'r5650421\'|\'315\'|\'25\'|\'Female\'|
|\'train\/Stasjon01\/070800\'|\'r5650083\'|\'\'|\'\'|\'\'|
|\'train\/Stasjon02\/280700\'|\'r5650204\'|\'194\'|\'21\'|\'Male\'|
|\'train\/Stasjon6\/140800\'|\'r5650562\'|\'575\'|\'50\'|\'Female\'|
|\'train\/Stasjon01\/310700\'|\'r5650002\'|\'1\'|\'23\'|\'Female\'|
|\'da\_0611\_test\/Stasjon05\/071000\_a\'|\'r6110033\'|\'21\'|\'30\'|\'Female\'|
|\'train\/Stasjon01\/280700\'|\'r5650048\'|\'44\'|\'39\'|\'Male\'|
|...|...|...|...|...|


In [86]:
speaker_counts = speaker_info_tab.table("select speaker_id, prefix, count(distinct age) as count from _ group by prefix, speaker_id")
speaker_counts.view("select * from _ order by count desc")

|speaker\_id|prefix|count|
|-|-|-|
|\'406\'|\'train\/Stasjon5\/070700\'|4|
|\'406\'|\'train\/Stasjon5\/100700\'|4|
|\'406\'|\'train\/Stasjon5\/110700\'|4|
|\'406\'|\'train\/Stasjon5\/120700\'|4|
|\'7\'|\'da\_0611\_test\/Stasjon05\/051000\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/071000\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/081000\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/08100\_b\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/110900\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/110900\_b\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/111000\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/120900\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/120900\_b\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/130900\_a\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/130900\_b\'|3|
|\'7\'|\'da\_0611\_test\/Stasjon05\/140900\_a\'|3|
|...|...|...|


In [88]:
speaker_info_tab.view("select distinct * from _ where speaker_id = '406'")

|prefix|recording|speaker\_id|age|sex|
|-|-|-|-|-|
|\'train\/Stasjon5\/110700\'|\'r5650013\'|\'406\'|\'67\'|\'Female\'|
|\'train\/Stasjon5\/120700\'|\'r5650012\'|\'406\'|\'43\'|\'Female\'|
|\'train\/Stasjon5\/070700\'|\'r5650011\'|\'406\'|\'\'|\'\'|
|\'train\/Stasjon5\/070700\'|\'r5650012\'|\'406\'|\'43\'|\'Female\'|
|\'train\/Stasjon5\/100700\'|\'r5650013\'|\'406\'|\'67\'|\'Female\'|
|\'train\/Stasjon5\/110700\'|\'r5650010\'|\'406\'|\'26\'|\'Female\'|
|\'train\/Stasjon5\/120700\'|\'r5650011\'|\'406\'|\'\'|\'\'|
|\'train\/Stasjon5\/110700\'|\'r5650012\'|\'406\'|\'43\'|\'Female\'|
|\'train\/Stasjon5\/120700\'|\'r5650010\'|\'406\'|\'26\'|\'Female\'|
|\'train\/Stasjon5\/100700\'|\'r5650011\'|\'406\'|\'\'|\'\'|
|\'train\/Stasjon5\/070700\'|\'r5650013\'|\'406\'|\'67\'|\'Female\'|
|\'train\/Stasjon5\/070700\'|\'r5650010\'|\'406\'|\'26\'|\'Female\'|
|\'train\/Stasjon5\/100700\'|\'r5650010\'|\'406\'|\'26\'|\'Female\'|
|\'train\/Stasjon5\/110700\'|\'r5650011\'|\'406\'|\'\'|\'\'|
|\'train\/Stasjon5\/120700\'|\'r5650013\'|\'406\'|\'67\'|\'Female\'|
|\'train\/Stasjon5\/100700\'|\'r5650012\'|\'406\'|\'43\'|\'Female\'|


In [89]:
counts = transcript_tab.table("select prefix, recording, audio_file, count(distinct text) as count from _ group by prefix, recording, audio_file")

In [90]:
counts.view("select * from _ order by count desc")

|prefix|recording|audio\_file|count|
|-|-|-|-|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'\-1\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001001\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001002\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001003\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001004\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001005\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001006\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001007\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001008\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001009\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001010\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001011\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001012\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001013\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001014\.wav\'|1|
|\'da\_0611\_test\/Stasjon03\/041000\'|\'r6110001\'|\'u0001015\.wav\'|1|
|...|...|...|...|


In [108]:
audio_tab

|prefix|recording|audio\_file|path|
|-|-|-|-|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512142\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512142\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512261\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512261\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512205\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512205\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512244\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512244\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512126\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512126\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512296\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512296\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512301\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512301\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512304\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512304\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512163\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512163\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512271\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512271\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512149\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512149\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512084\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512084\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512176\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512176\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512291\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512291\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512186\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512186\.wav\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512065\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512065\.wav\'|
|...|...|...|...|


In [109]:
%%time
annotated = audio_tab.table(
    """
select _.*, t.text, s.speaker_id from _ 
left join transcript as t using (prefix, recording, audio_file)
left join speaker_info as s using (prefix, recording)
    """,
    bindings={"transcript": transcript_tab, 
              "speaker_info": speaker_info_tab})

CPU times: user 17.8 s, sys: 23.7 s, total: 41.5 s
Wall time: 44.3 s


In [110]:
annotated

|prefix|recording|audio\_file|path|text|speaker\_id|
|-|-|-|-|-|-|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512142\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512142\.wav\'|\'Alt er lagt til rette\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512261\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512261\.wav\'|\'uddannet\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512205\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512205\.wav\'|\'Formanden for rådet glæder sig over\,\\\\Komma at der er taget hul på problemerne med generationsskifte\.\\\\Punktum\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512244\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512244\.wav\'|\'Herning\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512126\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512126\.wav\'|\'elleve tusind og seksogtyve\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512296\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512296\.wav\'|\'liste\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512301\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512301\.wav\'|\'telefon privat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512304\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512304\.wav\'|\'optag notat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512163\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512163\.wav\'|\'Det betyder\, at når solen opvarmer væsken i solfangeren\, opstår der en forskel i massefylde mellem væsken i solfangeren og i beholderen\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512271\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512271\.wav\'|\'forklare\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512149\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512149\.wav\'|\'Det er de traditionelle druer\, der fortrinsvis anvendes\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512084\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512084\.wav\'|\'Det stigende behov for kraftvarme skyldes at tilslutningen til det kollektive varmesystem er øget betydeligt og fortsat vil øges\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512176\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512176\.wav\'|\'De fleste af aktiviteterne har ikke gennem de senere år genereret en indtjening\, der bare tilnærmelsesvis har kunnet modsvare omkostningerne\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512291\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512291\.wav\'|\'hastighed\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512186\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512186\.wav\'|\'Det gik slag i slag\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512065\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512065\.wav\'|\'Han var den første\, der skaffede diasfilm til landet efter krigen\, og hans forretning udviklede sig til en stor\'|\'509\'|
|...|...|...|...|...|...|


In [111]:
annotated.view("select distinct prefix, recording from _ where text is null")

|prefix|recording|
|-|-|
|\'train\/Stasjon6\/260700\'|\'r5650524\'|
|\'train\/Stasjon03\/210700\'|\'r5650057\'|
|\'train\/Stasjon03\/140700\'|\'r5650033\'|
|\'train\/Stasjon03\/040800\'|\'r5650279\'|
|\'train\/Stasjon4\/210700\'|\'r5650461\'|
|\'train\/Stasjon4\/030800\'|\'r5650490\'|
|\'train\/Stasjon01\/140700\'|\'r5650048\'|
|\'train\/Stasjon5\/030800\'|\'r5650096\'|
|\'da\_0611\_test\/Stasjon03\/061000\'|\'r6110051\'|
|\'da\_0611\_test\/Stasjon03\/240900\_1\'|\'r6110042\'|
|\'da\_0611\_test\/Stasjon03\/180900\'|\'r6110005\'|
|\'da\_0611\_test\/Stasjon05\/080900\_a\'|\'r6110013\'|
|\'da\_0611\_test\/Stasjon05\/060900\'|\'r6110005\'|


In [28]:
transcript_tab.view("select * from _ where prefix = 'da_0611_test/Stasjon03/061000' and recording='r6110051' and audio_file like 'u002%'")

|prefix|recording|audio\_file|text|
|-|-|-|-|


In [112]:
db.execute("attach '/data/nst/annotations.sqlite3' as annotations2")

In [113]:
db.create_table(name="annotations2.annotations", column_names=iter(annotated).column_names, rows=annotated)

|prefix|recording|audio\_file|path|text|speaker\_id|
|-|-|-|-|-|-|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512142\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512142\.wav\'|\'Alt er lagt til rette\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512261\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512261\.wav\'|\'uddannet\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512205\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512205\.wav\'|\'Formanden for rådet glæder sig over\,\\\\Komma at der er taget hul på problemerne med generationsskifte\.\\\\Punktum\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512244\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512244\.wav\'|\'Herning\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512126\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512126\.wav\'|\'elleve tusind og seksogtyve\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512296\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512296\.wav\'|\'liste\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512301\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512301\.wav\'|\'telefon privat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512304\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512304\.wav\'|\'optag notat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512163\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512163\.wav\'|\'Det betyder\, at når solen opvarmer væsken i solfangeren\, opstår der en forskel i massefylde mellem væsken i solfangeren og i beholderen\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512271\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512271\.wav\'|\'forklare\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512149\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512149\.wav\'|\'Det er de traditionelle druer\, der fortrinsvis anvendes\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512084\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512084\.wav\'|\'Det stigende behov for kraftvarme skyldes at tilslutningen til det kollektive varmesystem er øget betydeligt og fortsat vil øges\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512176\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512176\.wav\'|\'De fleste af aktiviteterne har ikke gennem de senere år genereret en indtjening\, der bare tilnærmelsesvis har kunnet modsvare omkostningerne\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512291\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512291\.wav\'|\'hastighed\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512186\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512186\.wav\'|\'Det gik slag i slag\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512065\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512065\.wav\'|\'Han var den første\, der skaffede diasfilm til landet efter krigen\, og hans forretning udviklede sig til en stor\'|\'509\'|
|...|...|...|...|...|...|


In [115]:
db.create_table(name="annotations2.transcripts", column_names=iter(transcript_tab).column_names, rows=transcript_tab)
db.create_table(name="annotations2.audio_files", column_names=iter(audio_tab).column_names, rows=audio_tab)
db.create_table(name="annotations2.speaker_info", column_names=iter(speaker_info_tab).column_names, rows=speaker_info_tab)

OperationalError: table transcripts already exists

In [116]:
db.create_table(name="annotations2.speaker_info", column_names=iter(speaker_info_tab).column_names, rows=speaker_info_tab)

|prefix|recording|speaker\_id|age|sex|
|-|-|-|-|-|
|\'da\_0611\_test\/Stasjon05\/240900\'|\'r6110012\'|\'3\'|\'25\'|\'Male\'|
|\'train\/Stasjon02\/020800\'|\'r5650189\'|\'182\'|\'67\'|\'Male\'|
|\'train\/Stasjon02\/200700\'|\'r5650057\'|\'152\'|\'65\'|\'Male\'|
|\'train\/Stasjon4\/240700\'|\'r5650420\'|\'314\'|\'36\'|\'Female\'|
|\'train\/Stasjon01\/240700\'|\'r5650046\'|\'41\'|\'56\'|\'Female\'|
|\'train\/Stasjon01\/030800\'|\'r5650020\'|\'16\'|\'26\'|\'Female\'|
|\'train\/Stasjon02\/030800\'|\'r5650182\'|\'175\'|\'58\'|\'Female\'|
|\'da\_0611\_test\/Stasjon03\/051000\'|\'r6110046\'|\'44\'|\'21\'|\'Male\'|
|\'train\/Stasjon4\/120700\'|\'r5650410\'|\'305\'|\'46\'|\'Male\'|
|\'train\/Stasjon4\/140800\'|\'r5650421\'|\'315\'|\'25\'|\'Female\'|
|\'train\/Stasjon01\/070800\'|\'r5650083\'|\'\'|\'\'|\'\'|
|\'train\/Stasjon02\/280700\'|\'r5650204\'|\'194\'|\'21\'|\'Male\'|
|\'train\/Stasjon6\/140800\'|\'r5650562\'|\'575\'|\'50\'|\'Female\'|
|\'train\/Stasjon01\/310700\'|\'r5650002\'|\'1\'|\'23\'|\'Female\'|
|\'da\_0611\_test\/Stasjon05\/071000\_a\'|\'r6110033\'|\'21\'|\'30\'|\'Female\'|
|\'train\/Stasjon01\/280700\'|\'r5650048\'|\'44\'|\'39\'|\'Male\'|
|...|...|...|...|...|


In [117]:
db.close()

In [118]:
import csv
with open("/data/nst/annotations.csv", "w") as f:
    writer = csv.writer(f)
    it = iter(annotated)
    writer.writerow(it.column_names)
    writer.writerows(annotated)

ProgrammingError: Cannot operate on a closed database.